In [76]:
# Filepaths to know of
# Standard
# config_filepath = f"/home/adelgior/afs_directories/espresso/code/multi-instance-mask-rcnn-extension/output/logs/train/train_2021-06-17-095941_VCS-2dbaca2_MAX_ITR-100000_HEAD_TYPE-None/config.yaml"
# model_checkpoint = f"/home/adelgior/afs_directories/espresso/code/multi-instance-mask-rcnn-extension/output/logs/train/train_2021-06-17-095941_VCS-2dbaca2_MAX_ITR-100000_HEAD_TYPE-None/model_checkpoints/model_000000.pth.tar"
# Custom
config_filepath = f"/home/adelgior/afs_directories/espresso/code/multi-instance-mask-rcnn-extension/output/logs/train/train_2021-06-16-154242_VCS-f165af5_MAX_ITR-100000_HEAD_TYPE-custom_MATCH-0/config.yaml"
model_checkpoint = f"/home/adelgior/afs_directories/espresso/code/multi-instance-mask-rcnn-extension/output/logs/train/train_2021-06-16-154242_VCS-f165af5_MAX_ITR-100000_HEAD_TYPE-custom_MATCH-0/model_checkpoints/model_000000.pth.tar"

impth = "/home/adelgior/afs_directories/kalman/data/datasets/coco2017/images/val2017/000000000139.jpg"


In [77]:
%cd /home/adelgior/afs_directories/kalman/code/multi-instance-mask-rcnn-extension/

/home/allie/afs_directories/kalman/code/multi-instance-mask-rcnn-extension


In [78]:
# noinspection PyUnresolvedReferences
from multimaskextension.model import multi_roi_heads_apd
# noinspection PyUnresolvedReferences
from multimaskextension.data import registryextension

import cv2
import os
## Later objectives:
# Use this on the Kitchen dataset
# Change the training loss to re-learn the instance generation.
import time
import torch.distributed
from detectron2.engine import DefaultPredictor
from detectron2.evaluation.evaluator import inference_context
import detectron2.data.transforms as T
import pickle

from multimaskextension.train import script_utils
from multimaskextension.train import script_utils
from multimaskextension.train.script_utils import get_maskrcnn_cfg, DETECTRON_REPO
from multimaskextension.analysis.vis_utils import cv2_imshow, FigExporter
from multimaskextension.train.trainer_apd import Trainer_APD

import local_pyutils

def dbprint(*args, **kwargs):
    print(*args, **kwargs)

def get_model_input(original_image):
    transform_gen = T.ResizeShortestEdge(
                [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
            )
    if cfg.INPUT.FORMAT == "RGB":
        # whether the model expects BGR inputs or RGB
        original_image = original_image[:, :, ::-1]
    height, width = original_image.shape[:2]
    image = transform_gen.get_transform(original_image).apply_image(original_image)
    image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
    inputs = {"image": image, "height": height, "width": width}
    return inputs

def visualize(im, outputs, name='prediction', mask_name='pred_masks'):
    # d. Visualize and export Mask R-CNN predictions
    from detectron2.utils.visualizer import Visualizer
    from detectron2.data import MetadataCatalog
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    outputs["instances"].pred_masks = getattr(outputs['instances'], mask_name)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])
    exporter.export_gcf(name)

def detach_outputs(outputs):
    for f in outputs['instances']._fields:
        t = getattr(outputs['instances'], f)
        if torch.is_tensor(t):
            if t.requires_grad:
                t = t.detach()
        else:
            if t.tensor.requires_grad:
                t.tensor = t.tensor.detach()
                
def activate_head_type(model, head_type):
    model.roi_heads.active_mask_head = head_type
    if head_type == 'custom':
        assert type(model.roi_heads.mask_heads[model.roi_heads.active_mask_head]) is \
               CustomMaskRCNNConvUpsampleHeadAPD, 'Not using custom head; head type is {}'.format(type(
            model.roi_heads.mask_heads[model.roi_heads.active_mask_head]))
    else:
        assert type(model.roi_heads.mask_heads[model.roi_heads.active_mask_head]) \
               is not CustomMaskRCNNConvUpsampleHeadAPD



In [79]:
# Model setup mimicking training behavior
from detectron2.checkpoint import DetectionCheckpointer
from multimaskextension.train.script_utils import Timer
from detectron2.modeling import build_model

def build_model_without_dataloader(cfg, checkpoint_file):
    with Timer('Building model'):
        model = build_model(cfg)

    checkpointer = DetectionCheckpointer(model)
    checkpointer.load(cfg.MODEL.WEIGHTS)

    transform_gen = T.ResizeShortestEdge(
        [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
    )

    input_format = cfg.INPUT.FORMAT
    assert input_format in ["RGB", "BGR"], input_format

    if checkpoint_file is not None:
        assert os.path.exists(checkpoint_file)
        state = torch.load(checkpoint_file)
        for key, value in state.items():
            if key == 'model_state_dict' or key == 'model':
                model.load_state_dict(value)
    return model
        
def build_model_with_dataloader(cfg, checkpoint_resume):
    output_dir = '/tmp/'
    trainer = Trainer_APD(cfg, out_dir=output_dir, interval_validate=1000, n_model_checkpoints=20,
                          checkpoint_resume=checkpoint_resume)
    return trainer.model
    


In [80]:

output_dir = '/tmp/'
resume_logdir = None
print('Running setup...')
cfg = script_utils.get_custom_maskrcnn_cfg(config_filepath)

head_type = cfg.MODEL.ROI_MASK_HEAD.INIT_ACTIVATED_MASK_HEAD \
    if cfg.MODEL.ROI_HEADS.NAME != 'StandardROIHeads' else None
config_dictionary = {'max_itr': cfg.SOLVER.MAX_ITER, 'head_type': head_type}
if head_type == 'custom':
    config_dictionary.update({'match': int(cfg.MODEL.ROI_MASK_HEAD.MATCHING_LOSS)})

if resume_logdir is not None:
    assert os.path.exists(os.path.join(output_dir, 'config.yaml'))
    config_outpath = os.path.join(output_dir, "config_resume.yaml")
else:
    config_outpath = os.path.join(output_dir, "config.yaml")

with open(config_outpath, "w") as f:
    f.write(cfg.dump())
print("Full config saved to {}".format(os.path.abspath(config_outpath)))
checkpoint_resume = None if resume_logdir is None else os.path.join(resume_logdir, rel_model_pth)



Running setup...
Full config saved to /tmp/config.yaml


In [81]:
SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model1 = build_model_without_dataloader(cfg, checkpoint_resume)

SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model2 = build_model_without_dataloader(cfg, checkpoint_resume)

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)
# model2 = build_model_with_dataloader(cfg, checkpoint_resume)

[Building model]
Elapsed: 0.4307868480682373
[Building model]
Elapsed: 0.47775721549987793
All states same between modela and model b


In [82]:
SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model1 = build_model_without_dataloader(cfg, checkpoint_resume)

SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model2 = build_model_without_dataloader(cfg, checkpoint_resume)

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)
print((modela.state_dict()['roi_heads.custom_mask_head.predictor.weight'] == modelb.state_dict()['roi_heads.custom_mask_head.predictor.weight']).sum(), modelb.state_dict()['roi_heads.custom_mask_head.predictor.weight'].numel())
# model2 = build_model_with_dataloader(cfg, checkpoint_resume)


[Building model]
Elapsed: 0.43318629264831543
[Building model]
Elapsed: 0.43639445304870605
All states same between modela and model b
tensor(40960, device='cuda:0') 40960


In [83]:
modela.state_dict()['roi_heads.custom_mask_head.predictor.weight'].shape

torch.Size([160, 256, 1, 1])

In [84]:
for k in [k for k in modela.state_dict() if k.startswith('roi_heads')]:
    print(k, '\t\t', modela.state_dict()[k].shape)

roi_heads.box_head.fc1.weight 		 torch.Size([1024, 12544])
roi_heads.box_head.fc1.bias 		 torch.Size([1024])
roi_heads.box_head.fc2.weight 		 torch.Size([1024, 1024])
roi_heads.box_head.fc2.bias 		 torch.Size([1024])
roi_heads.box_predictor.cls_score.weight 		 torch.Size([81, 1024])
roi_heads.box_predictor.cls_score.bias 		 torch.Size([81])
roi_heads.box_predictor.bbox_pred.weight 		 torch.Size([320, 1024])
roi_heads.box_predictor.bbox_pred.bias 		 torch.Size([320])
roi_heads.standard_mask_head.mask_fcn1.weight 		 torch.Size([256, 256, 3, 3])
roi_heads.standard_mask_head.mask_fcn1.bias 		 torch.Size([256])
roi_heads.standard_mask_head.mask_fcn2.weight 		 torch.Size([256, 256, 3, 3])
roi_heads.standard_mask_head.mask_fcn2.bias 		 torch.Size([256])
roi_heads.standard_mask_head.mask_fcn3.weight 		 torch.Size([256, 256, 3, 3])
roi_heads.standard_mask_head.mask_fcn3.bias 		 torch.Size([256])
roi_heads.standard_mask_head.mask_fcn4.weight 		 torch.Size([256, 256, 3, 3])
roi_heads.standard_mas

In [ ]:
SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model1 = build_model_without_dataloader(cfg, checkpoint_resume)

SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model2 = build_model_with_dataloader(cfg, checkpoint_resume)

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)
# model2 = build_model_with_dataloader(cfg, checkpoint_resume)

In [ ]:
SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model1 = build_model_without_dataloader(cfg, checkpoint_resume)

SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model2 = build_model_without_dataloader(cfg, checkpoint_resume)

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)
# model2 = build_model_with_dataloader(cfg, checkpoint_resume)

In [ ]:
SEED = 0
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model1 = build_model_without_dataloader(cfg, checkpoint_resume)

SEED = 50
import torch
torch.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

model2 = build_model_with_dataloader(cfg, checkpoint_resume)

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)
# model2 = build_model_with_dataloader(cfg, checkpoint_resume)

In [ ]:
# build_model_without_dataloader produces the same model state dictionary regardless of the random seed set.
# build_model_without_dataloader produces the same model state dictionary when random seed it set the same.
#'roi_heads.custom_mask_head.predictor.weight'

modela = model1
modelb = model2

assert set(modela.state_dict().keys()) == set(modelb.state_dict().keys())
mismatched_states = []
for k in modela.state_dict().keys():
    if not (modela.state_dict()[k] == modelb.state_dict()[k]).all():
        mismatched_states.append(k)
#         print(k)
if len(mismatched_states) == 0:
    print('All states same between modela and model b')
else:
    print('Mismatched states')
    print(mismatched_states)

In [ ]:
(modela.state_dict()['roi_heads.custom_mask_head.predictor.weight'] == modelb.state_dict()['roi_heads.custom_mask_head.predictor.weight']).sum()

In [ ]:
type(model)

In [ ]:
model = model1

if not cfg.MODEL.ROI_HEADS.NAME == 'StandardROIHeads':
    activate_head_type(model, head_type)

# I. Load pre-existing Mask R-CNN model
# a. Get example image to work with
cpu = 1

im = cv2.imread(impth)
cv2_imshow(im)
exporter = FigExporter(workspace_dir='/home/allie/workspace/images/')
exporter.export_gcf('input')

# c. Get Mask R-CNN predictions (and print boxes)
inputs = get_model_input(im)
outputs = model([inputs])[0]
detach_outputs(outputs)

original_image = im
mask_name = 'pred_masks1'
# d. Visualize and export Mask R-CNN predictions
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
outputs["instances"].pred_masks = getattr(outputs['instances'], mask_name)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])
exporter.export_gcf(name)

# visualize(im, outputs, 'pred_masks1')


In [ ]:
# 'model_final_f10217.pkl'
initialization_file = os.path.join('/home/adelgior/afs_directories/espresso/data/models/detectron_model_zoo/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/', os.path.basename(cfg.MODEL.WEIGHTS))

print('Loading state dict')
cpt_state = torch.load(model_checkpoint, map_location=torch.device('cpu')) if cpu \
    else torch.load(model_checkpoint)
init_state = pickle.load(open(initialization_file, 'rb'))
init_state['model'] = {k: torch.Tensor(v) for k, v in init_state['model'].items()}
# model_state = cpt_state['model_state_dict']
model_state = init_state['model']
model.load_state_dict(model_state)
model.load_checkpoint()


In [ ]:
# cfg = get_maskrcnn_cfg(config_filepath)
model = Trainer_APD.build_model(cfg)
model.eval()
print('Loading state dict')
cpt_state = torch.load(model_checkpoint, map_location=torch.device('cpu')) if cpu \
    else torch.load(model_checkpoint)
init_state = pickle.load(open(initialization_file, 'rb'))
init_state['model'] = {k: torch.Tensor(v) for k, v in init_state['model'].items()}
# model_state = cpt_state['model_state_dict']
model_state = init_state['model']
model.load_state_dict(model_state)
model.load_checkpoint()


In [ ]:
# # cfg = get_maskrcnn_cfg(config_filepath)
# cfg.MODEL.DEVICE = 'cpu'
# model = Trainer_APD.build_model(cfg)
# model.eval()


In [ ]:
# from pprint import pprint
# pprint(outputs["instances"].pred_classes)
# pprint(outputs["instances"].pred_boxes)

